In [1]:
import pprint
import os, json
import pandas as pd 
pd.set_option('display.max_rows', 500)

In [2]:
roles = [
        'Business Analyst',
        'Backend Engineer',
        'DevOps Engineer',
        'Frontend Engineer',
        'FullStack Engineer',
        'Project Manager',
        'Quality Assurance Engineer', 'Tech Lead'
        ]

In [3]:
def load_json_files(data_path = 'data/KPI/jsons/'):
    json_arr = {}
    files = os.listdir(data_path)
    for file in files:
        if file.endswith('.json'):
            json_path = os.path.join(data_path, file)
            with open(json_path, 'r') as f:
                data = json.load(f)
                data_updated = {}
                for type, criteria in data.items():
                    data_iter = {}
                    for cri, levels in criteria.items():
                        cri = cri.replace('/', '-')
                        data_iter[cri] = levels
                    data_updated[type] = data_iter

                role = file.split('.')[0]
                json_arr[role] = [data_updated, json_path]
    return json_arr

In [4]:
json_arr = load_json_files()
pprint.pprint(json_arr)

{'Backend Engineer': [{'education': {"Bachelor's Degree": {'Unrelated': 5,
                                                           'related': 10},
                                     "Master's Degree": {'Unrelated': 5,
                                                         'related': 10}},
                       'experience': {'Experience of related Domain': {'0 - 5': 3,
                                                                       '15+': 10,
                                                                       '6 - 14': 6},
                                      'Years of experience in Bacend Engineer': {'1-2 years': 2,
                                                                                 '3-5 years': 4,
                                                                                 '5+ years': 6.6}},
                       'skills': {'API Development and Integration': {'Advanced': 10,
                                                                      'In

In [5]:
def load_csv_files(data_path = 'data/KPI/weights/'):
    df_arr = {}
    files = os.listdir(data_path)
    for file in files:
        if file.endswith('.xlsx'):
            excel_path = os.path.join(data_path, file)
            with open(excel_path, 'r') as f:
                df = pd.read_excel(excel_path)
                df = df[['Criteria', 'Weight']]
                role = file.split('.')[0]
                df_arr[role] = df
    return df_arr

In [6]:
df_arr = load_csv_files()
df_arr

{'Quality Assurance Engineer':                              Criteria  Weight
 0            Excellent communication     0.05
 1                     Test Automation    0.20
 2  Knowledge of testing methodologies    0.15
 3          Bug tracking and reporting    0.10
 4           Years of experience in QA    0.15
 5        Experience of related Domain    0.10
 6     Leadership/Team lead experience    0.05
 7                   Bachelor's Degree    0.10
 8                     Master's Degree    0.10,
 'Frontend Engineer':                                             Criteria  Weight
 0                            Proficiency in HTML/CSS    0.15
 1               Proficiency in JavaScript/TypeScript    0.10
 2         Knowledge of Frontend Frameworks/Libraries    0.15
 3                            UI/UX Design Principles    0.10
 4  Responsive Design and Cross-Browser Compatibility    0.05
 5           Years of experience in FrontEnd engineer    0.15
 6                       Experience of relat

In [7]:
crud_json = {
        "type" : "skills",
        "criteria" : "Analytical Skills KKK",
        "level" : {
                "Novice":1,
                "Intermediate":3,
                "Advanced":5
                },
        "weight" : 0.20
        }

In [8]:
def crud_kpi_criterias(
                        crud_json, role, 
                        operation = 'add' # ['add', 'delete', 'update']
                        ):
    try:
        json_arr = load_json_files()
        df_arr = load_csv_files()
        if operation == 'add':
            df_role = df_arr[role]
            if crud_json['criteria'] in df_role['Criteria'].values:
                return "Criteria Already Exists !!!"
            
            df_weights = pd.DataFrame({
                                    'Criteria' : [crud_json['criteria']],
                                    'Weight' : [crud_json['weight']]
                                    })
            df_role = pd.concat([df_role, df_weights], axis = 0)
            
            json_role, json_role_path = json_arr[role]
            json_role[crud_json["type"]][crud_json["criteria"]] = crud_json["level"]


        elif operation == 'delete':
            df_role = df_arr[role]
            if crud_json['criteria'] not in df_role['Criteria'].values:
                return "Criteria Does Not Exists !!!"
            
            df_role = df_role[df_role['Criteria'] != crud_json['criteria']]
            
            json_role, json_role_path = json_arr[role]
            del json_role[crud_json["type"]][crud_json["criteria"]]
            

        elif operation == 'update':
            df_role = df_arr[role]
            if crud_json['criteria'] not in df_role['Criteria'].values:
                return "Criteria Does Not Exists !!!"
            
            df_role.loc[df_role['Criteria'] == crud_json['criteria'], 'Weight'] = crud_json['weight']
            
            json_role, json_role_path = json_arr[role]
            json_role[crud_json["type"]][crud_json["criteria"]] = crud_json["level"]
            

        with open(json_role_path, 'w') as f:
            json.dump(json_role, f)

        # recalculate the weights
        df_role['Weight'] = df_role['Weight'] / df_role['Weight'].sum()
        df_role.to_excel(
                        f'data/KPI/weights/{role}.xlsx', 
                        index = False
                        )
        return "CRUD Operation Successful !!!"

    except Exception as e:
        return str(e)

In [21]:
def apply_kpi_level(
                    row, 
                    json_role_updated=None
                    ):
    weight = row['Weight']
    criteria = row['Criteria'].replace('\\/', '-').replace('\\', '').replace('\/', '-')
    level = row['Level']

    value = json_role_updated[criteria][level]
    return value * weight


def calculate_kpi_value(
                        role,
                        criteria_json
                        ):
    df_arr = load_csv_files()
    json_arr = load_json_files()

    df_role = df_arr[role]
    df_role['Criteria'] = df_role['Criteria'].str.replace('\\/', '-').str.replace('\\', '').str.replace('/', '-')
    json_role = json_arr[role][0]

    json_role_updated = {}
    for _, value in json_role.items():
        for k, v in value.items():
            json_role_updated[k] = v

    criteria_df = pd.DataFrame(
                                criteria_json, 
                                index=[0]
                                ).T
    criteria_df = criteria_df.reset_index()
    criteria_df.columns = ['Criteria', 'Level']
    criteria_df['Criteria'] = criteria_df['Criteria'].str.strip()
    criteria_df['Level'] = criteria_df['Level'].str.strip()
    criteria_df = criteria_df.merge(
                                    df_role, 
                                    on = 'Criteria', 
                                    how = 'left'
                                    )
    criteria_df['Weight'] = criteria_df['Weight'].fillna(0)
    criteria_df['KPI'] = criteria_df.apply(apply_kpi_level, axis = 1, json_role_updated=json_role_updated)
    kpi_value = criteria_df['KPI'].sum()
    return kpi_value

def calculate_kpi_sheet(
                        role, domain,
                        employee_file_path = 'data/KPI/employees.xlsx'
                        ):
    df_kpi = {}
    df_kpi['EmpID'] = []
    df_kpi['Domain'] = []
    df_kpi['Role'] = []
    df_kpi['KPI'] = []

    df_role_values = pd.read_excel(
                                    employee_file_path,
                                    sheet_name=role
                                    )
    df_role_values = df_role_values[df_role_values['Domain'] == domain]
    df_role_values.reset_index(drop=True, inplace=True)

    for i in range(df_role_values.shape[0]):
        criteria_json = eval(df_role_values.loc[i, :].to_json())
        criteria_json = {k.replace('\\/', '-').replace('\\', '').replace('\/', '-'): v for k, v in criteria_json.items()}
        emp_id = criteria_json['EMP ID']
        domain = criteria_json['Domain']
        del criteria_json['EMP ID'], criteria_json['Domain']
        kpi_value = calculate_kpi_value(
                                        role,
                                        criteria_json
                                        )

        df_kpi['KPI'].append(kpi_value)
        df_kpi['Domain'].append(domain)
        df_kpi['EmpID'].append(emp_id)
        df_kpi['Role'].append(role)

    df_kpi = pd.DataFrame(df_kpi)
    df_kpi = df_kpi[[
                    'EmpID',
                    'KPI'
                    ]]
    return df_kpi

In [22]:
criteria_json = {
                "Analytical Skills":"Advanced ",
                "Technical Proficiency":"Advanced ",
                "Communication Skills":"Advanced ",
                "Problem Solving Skills":"Advanced ",
                "Years of experience in Business Analysis":"5+ years",
                "Experience of related Domain":"15+ ",
                "Leadership-Team lead experience":"Leadership",
                "Bachelor's Degree":"related",
                "Master's Degree":"related"
                }
calculate_kpi_value('Business Analyst', criteria_json)

10.0

In [23]:
calculate_kpi_sheet('Business Analyst', 'Finance')

,EmpID,KPI
0,BA1,10.00
1,BA2,7.20
2,BA3,3.00
3,BA4,7.80
4,BA5,4.85
5,BA6,7.10
6,BA7,5.60
7,BA8,7.05
8,BA9,7.35
9,BA10,5.95
